In [1]:
import pandas as pd

In [2]:
import geopandas as gpd

In [3]:
import math
import osmnx as ox

In [4]:
import shapely as sh

In [5]:
segments_data = pd.read_csv('segments.csv')

In [6]:
def get_coord(coords, inverted=False):
    if inverted:
        return sh.geometry.LineString([(c[1],c[0]) for c in coords])    
    return sh.geometry.LineString([(c[0],c[1]) for c in coords])

In [7]:
segments_data['geometry']=segments_data['route_wkt'].apply(lambda x: get_coord(sh.wkt.loads(x).coords))

In [8]:
segments_data = segments_data.drop(columns=['route_wkt'])

In [9]:
segments = gpd.GeoDataFrame(segments_data)

In [10]:
walking = segments[segments['modal_type']=='Walking'].copy()

In [11]:
walking = walking.sort_values('distance_travelled', ascending=False)

In [12]:
walking = walking.drop_duplicates('route_id', keep='first')

In [13]:
walking['velocity']=walking.distance_travelled/(walking.time_travelled/3600)

In [14]:
walking_enough = walking[(walking.distance_travelled>0.5)&(walking['velocity']<6)]

In [98]:
def save_osmnx_network(route_data, plot=False):
    route = route_data.geometry
    polygon = route.buffer(0.0003)
    G_route = ox.graph_from_polygon(polygon, network_type='walk')
    G_e = ox.graph_to_gdfs(G_route, nodes=False)
    G_e.to_pickle('edges/{}.gpkl'.format(route_data.route_id))
    G_n = ox.graph_to_gdfs(G_route, edges=False)
    G_n.to_pickle('nodes/{}.gpkl'.format(route_data.route_id))
    return G_route

In [58]:
def quick_near_edges(G_route, x, y):
    edges = []
    nearest_nodes = ox.get_nearest_nodes(G_route, x, y)
    for i, node in enumerate(nearest_nodes):
        if i==len(nearest_nodes)-1:
            break
        shortest_path = ox.nx.shortest_path(G_route, nearest_nodes[i], nearest_nodes[i+1])
        if len(shortest_path)>=2 and len(shortest_path)<=5:
            for i, e in enumerate(shortest_path):
                if i>0:
                    edges.append(shortest_path[i-1:i+1])
    return edges

In [101]:
def save_geojson(G_route, plot=False):
    route = route_data.geometry
    polygon = route.buffer(0.0005)
    G_route = ox.graph_from_polygon(polygon, network_type='walk')
    x,y = route.coords.xy
    nearest_edges = quick_near_edges(G_route, x, y)
    if plot:
        m=ox.plot_graph_folium(ox.geo_utils.induce_subgraph(G_route,np.hstack(nearest_edges)))
        plot_points(get_coord(route.coords,inverted=True).coords , m)
        return m
    edge_attributes = ox.graph_to_gdfs(G_route, nodes=False)
    edge_attributes = edge_attributes[['u','v','osmid','geometry', 'name']].set_index(['u', 'v'])
    good_edges = edge_attributes.loc[[tuple(l) for l in nearest_edges]]
    good_edges.osmid = good_edges.osmid.apply(lambda x: '-'.join([str(idx) for idx in x]) if type(x) == list else str(x))
    good_edges.name = good_edges.name.apply(lambda x: '-'.join([str(idx) for idx in x]) if type(x) == list else str(x))
    good_edges = good_edges[~good_edges.index.duplicated(keep='first')].copy()
    good_edges['color']=0
    good_edges[['name','geometry','color']].to_file('shapes/{}.geojson'.format(route_data.route_id),driver='GeoJSON',  encoding='utf_8')

In [106]:
part_2 = walking_enough[int(len(walking_enough)/2)+21:]

In [107]:
errors = []

In [ ]:
for i, route_data in tqdm.tqdm(part_2.iterrows(),  total=part_2.shape[0]):
    try:
        save_geojson(route_data)
    except:
        errors.append(i)

 14%|█████████▉                                                            | 1390/9814 [12:05:41<296:03:34, 126.52s/it]